Modules installation and configuration

In [13]:
import sys
##Change the path to folder where your SPM data locates
sys.path.append(r'C:\Users\Yalan\PycharmProjects\GrainBot_v2')
sys.path.append(r'C:\Users\Yalan\PycharmProjects\GrainBot_v2\GBGchar')
from importlib.machinery import SourceFileLoader
import postprocess_special as process
import mask_operation as mask_op
#import GBGchar.CNNseg as seg
import AFMOpr
import MaskOpr
import GrainOpr
import StatisticM
##Install the following packages using pip first
import matplotlib.pyplot as plt
import random
import cv2
import numpy as np
import pandas as pd
segment=SourceFileLoader("ImageSeg", "ImageSeg.py").load_module()

Loading SPM images and seg

In [14]:
##Make sure there are only .png images need segmentation in this path
##Remove all the outputs before rerunning the code

imgpath=r'C:\Users\Yalan\GB-project-data\Validationset'
wei_path='params/unet.pth'
Dataset=segment.validation(wei_path)
Dataset.valid(imgpath)
##Successfully segmented images could be found in the same path


successfully segmented


UnidentifiedImageError: cannot identify image file 'C:\\Users\\Yalan\\GB-project-data\\Validationset\\a02.png.csv'

Microstructural parameters measurement

In [15]:
pddata=pd.DataFrame({'gsa':[1],'cve':[1],'rel_cve':[1],\
                     'rel_h':[1],'cca':[1],'rel_d':[1],'mean_angle':[1]})
pd_angle=pd.DataFrame({'grain number':[1],'gsa':[1]})
##Input scale information of SPM images (unit:nm)
z_min=-40
z_max=40
img_width=4000
for n in Dataset.filenames:
    afm_img=cv2.imread(imgpath+'\\'+n)
    mask_img=cv2.imread(imgpath+'\\'+'seg_'+n)
    act_img=AFMOpr.AFMimg(afm_img,mask_img,z_min,z_max,img_width)
    act_img.MorphOpr()
    act_img.Labeling()
    for i in range(1,act_img.numx):
    
        grain=GrainOpr.Grains(i)
        grain.grainmask(act_img.labelimg,act_img.afm)
        grain.maskdil(act_img.afm)
        grain.maskregain(act_img.afm)
        grain.maskero(act_img.afm)
        grain.areacal()
        volcoe=(act_img.lmax-act_img.lmin)/255*act_img.pixelsize*act_img.pixelsize
        areacoe=act_img.pixelsize*act_img.pixelsize
        if grain.area>50:
            grain.voidcal(5)##The step of sections used for CCA or CVE calculation
            grain.anglecal(act_img.realdimg,act_img.pixelsize)
        
            realarea=grain.area*areacoe
            data_i=pd.DataFrame({'gsa':[realarea],'cve':[grain.vb*volcoe],\
                                 'rel_cve':[grain.rel_vb*volcoe],'rel_h':[grain.vb*volcoe/realarea],\
                                 'cca':[grain.vd*volcoe],'rel_d':[grain.vd*volcoe/realarea],\
                                 'mean_angle':[max(grain.angles)]},index=[i])
            for j in range(0,len(grain.angles)):
                angle_j=pd.DataFrame({'grain number':[i],'gsa':[grain.angles[j]]},index=[])
                pd_angle=pd.concat([pd_angle,angle_j])
            
            pddata=pd.concat([pddata,data_i])
        else:
            continue
        
    pddata.to_csv(imgpath+'//'+n+'.csv')
    pd_angle.to_csv(imgpath+'//angles_'+n+'.csv')

error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


Statisitcal analysis on microstructural parameters

In [ ]:

Angledata=StatisticM.StatisticAng(imgpath+'//'+'allangles_A3.0_00007.csv')
CCAdata=StatisticM.StatisticCCA(imgpath+'//'+'A3.0_00002.csv')
CVEdata=StatisticM.StatisticCVE(imgpath+'//'+'A3.0_00002.csv')
GSAdata=StatisticM.StatisticGSA(imgpath+'//'+'A3.0_00002.csv')
figure = plt.figure(figsize=(12,10),edgecolor="black",frameon=True)
plt.tight_layout()
ax=plt.subplot(2,2,1)
Angledata.histplot(1,40)
plt.text(0.55, 0.7, 'mean: ' + '%.2f' % Angledata.mu, transform=ax.transAxes, size=18)
plt.text(0.55, 0.6, 'variance: ' + '%.2f' % Angledata.sigma, transform=ax.transAxes,size=18)
plt.subplot(2,2,2)
CCAdata.histplot()
plt.subplot(2,2,3)
CVEdata.histplot()
plt.subplot(2,2,4)
GSAdata.histplot()
plt.tight_layout(pad=2,w_pad=3)
